<CENTER>
</br>
<p><font size="5"> Nathan Sanglier </span></p>
<p><font size="4">  Stochastic Optimization </font></p>
<p></p>
<p><font size="5"> LMS Equalization for Digital Data Recovery </font></p>
</p></br>
</p>
</CENTER>

----------------------------

## <center>  <span style="color:#FF0000"> NOT FINISHED YET </span> </center>

##  <span style="color:#00B8DE"> 0 - Introduction </span>

> The goal of this notebook is to handle LMS equalization to recover BPSK symbols emitted which went through a propagation channel and were subject to noise. Thus, we will build a filter (LMS filter, which is built through stochastic optimization) to which we will convolve the received signal in order to recover the emitted symbols. If you want to see the plots and LaTeX equations appear well, please download and run the notebook.

# <span style="color:#00B8DE"> LMS equalization for digital data recovery </span>


> - We will enerate $n_s=10^4$ symbols of a BPSK modulation.
> - We will convolve symbols with a filter with impulse response given by vector ${\bf h} \propto [1, .8, .3]$ and $\parallel {\bf h}\parallel = 1$. This filter represents the effect of propagation channel on transmitted symbols.
> - We will add noise so as to achieve a fixed SNR at the receiver side.
> - We will implement an LMS filter with impulse response ${\bf w}$ of length $l_w=10$.
> - We will study the convergence of the filter, depending on the SNR. 
> - We will train the filter with $n_s=10^4$ symbols, and then generate data for $n_s=10^6$ symbols and $SNR=15dB$.
> - We will deconvolve the sequence and estimate the error probability $P_e$.
> - We will analyze the influence of the different parameters (stepsize, SNR, impulse response ${\bf h}$, length of the training sequence, ...) on the behavior of the algorithm. 

In [ ]:
using Plots
using LinearAlgebra
using Statistics

## <span style="color:#00B8DE"> Data generation </span>

> Let us generate $n_s=10^4$ symbols of a BPSK modulation.

In [ ]:
function generate_bpsk_symbols(ns)

    bpsk_symbols = rand([-1, 1], ns)    # The BPSK symbols are either 1 or -1

    return bpsk_symbols
end;

In [ ]:
ns = 10^4
bpsk_symbols = generate_bpsk_symbols(ns);

> Let us convolve symbols with a filter with impulse response given by vector ${\bf h} \propto [1, .8, .3]$ and $\parallel {\bf h}\parallel = 1$. This filter represents the effect of propagation channel on transmitted symbols.

In [ ]:
function convolve(x, h)     # Utility function to perform convolution : y = h ⋆ x = sum h_(k) * x_(n-k+1)

    y = zeros(length(x))

    for n in 1:length(x)
        for k in 1:length(h)
            if n - k + 1 > 0
                y[n] += h[k] * x[n - k + 1]
            end
        end
    end

    return y
end;

In [ ]:
function generate_signal(bpsk_symbols, h)       # Generate the transmitted signal

    return convolve(bpsk_symbols, h)
end;

In [ ]:
h = [1, 0.8, 0.3]
h /= norm(h)

signal = generate_signal(bpsk_symbols, h);

> Let us add noise so as to achieve a fixed SNR at the receiver side, and plot the values of the first 20 symbols and that of the first 20 data samples as well as their sign for $SNR=20dB$.

In [ ]:
function add_noise(signal, SNR)     # Add noise to the transmitted signal to create the signal received

    sigma = sqrt(10^(-SNR/20))
    noise = sigma * randn(length(signal))

    noisy_signal = signal + noise

    return noisy_signal
end;

In [ ]:
SNR = 20

noisy_signal = add_noise(signal, SNR);

In [ ]:
plot(1:20, bpsk_symbols[1:20], marker=:circle, label="BPSK Symbols", xlabel="Sample", ylabel="Amplitude", title="Symbols accross transmission", legend=:bottomright, ylims=(-3, 3))
plot!(1:20, noisy_signal[1:20], marker=:circle, label="Received Signal")
plot!(1:20, sign.(noisy_signal[1:20]), marker=:circle, label="Predicted Symbols (threshold)")

<span style="color:#DAF7A6">
As we can see, the effect of propagation channel on transmitted BPSK symbols and the adding of a gaussian noise lead to the missclassification of some symbols (based on the corresponding received value sign i.e. +1 if the received value is > 0 and -1 otherwise). Thus, we will have a not negligible error rate if we do not correct this problem.

> Let us check that even in the absence of noise the error probability $P_e$ resulting from decisions based on data thresholding would not be zero, and let us estimate the value of $P_e$ in the absence of equalization.

In [ ]:
function threshold_pred_symbols(signal)     # We predict the symbols based on the corresponding received value sign i.e. +1 if the received value is > 0 and -1 otherwise

    pred_symbols = sign.(signal)

    return pred_symbols
end;

In [ ]:
function calculate_error_proba(bpsk_symbols, pred_symbols)      # Empirical Error Probability (i.e. Error Rate)

    error_proba = sum(pred_symbols .!= bpsk_symbols) / length(bpsk_symbols)
end;

In [ ]:
pred_symbols_no_noisy = threshold_pred_symbols(signal)      # Symbols predicted when there is no gaussian noise in the received signal
pred_symbols_noisy = threshold_pred_symbols(noisy_signal)   # Symbols predicted when there is gaussian noise in the received signal


error_proba_no_noisy = calculate_error_proba(bpsk_symbols, pred_symbols_no_noisy)
error_proba_noisy = calculate_error_proba(bpsk_symbols, pred_symbols_noisy)

println("Error Probability (Pe) with noise in the received signal : $(round(error_proba_noisy*100,digits=2)) %")
println("Error Probability (Pe) without noise in the received signal : $(round(error_proba_no_noisy*100,digits=2)) %")

<span style="color:#DAF7A6">
As we can see,  even in the absence of noise, the error probability resulting from decisions based on data thresholding would not be zero. <br/>
Moreover, here the error rate is always higher when we do not have noise in our signal, it is certainly due to side effects of the gaussian noise for values that are near 0, and cancels sometimes the effect of the propagation channel.

## <span style="color:#00B8DE"> Definition & Convergence LMS Filter </span>

> Let us implement an LMS filter with impulse response ${\bf w}$ of length $l_w=10$. The stepsize is set to half the well-known maximum suggested stepsize and $SNR=20dB$.

In [ ]:
# Gradient of the objective function (changing with time)
∇f(signal, bpsk_symbol, θ, n, length_filter) = reverse(signal[n-length_filter+1 : n]) * (θ' * reverse(signal[n-length_filter+1 : n]) - bpsk_symbol);

In [ ]:
# LMS Algorithm
function stoch_gradient_lms(signal, length_filter, bpsk_symbols, μ, init_θ, ∇f)

    θ = init_θ  # θ is the estimation of coefficients of the filter at time n
    res = []    # Contains the estimations of the filter coefficients at each time 
    
    for n in length_filter:length(signal)   # n begins at length_filter, otherwise the gradient objective function is not defined
        
        direction = - ∇f(signal, bpsk_symbols[n], θ, n, length_filter)
        θ += μ * direction      # μ is the stepsize
        push!(res, θ)
    end

    return res
end;

In [ ]:
length_filter = 10                                  # length of the filter impulse response
max_μ = 1 / (length_filter * var(noisy_signal))     # Maximum stepsize that does not lead to the divergence of the LMS Algorithm
μ = 0.5 * max_μ                                     # We set the stepsize to half the maximum stepsize
init_θ = randn(length_filter)                       # We init the coefficients with random realizations of a standard gaussian law  

res = stoch_gradient_lms(noisy_signal, length_filter, bpsk_symbols, μ, init_θ, ∇f)
θ_lms = res[end];

> Let us plot the evolution of the distance $\parallel {\bf w}_{MSE}-{\bf w}_{LMS}\parallel $  between the $MSE$ estimate ${\bf w}_{MSE}$ of ${\bf w}$ and successsive LMS estimates $
{\bf w}_{LMS}$.

In [ ]:
# Minimum MSE Solution
function min_mse_filter(signal, length_filter, bpsk_symbols)

    R = zeros(length_filter, length_filter)
    r = zeros(length_filter)
    
    for n in length_filter:length(signal)
        
        array_signal_n = reverse(signal[n-length_filter+1 : n]) # samples of the signal from n to n-length_filter+1

        R += array_signal_n * array_signal_n'
        r += array_signal_n * bpsk_symbols[n]
    end

    R /= (length(signal) - length_filter + 1)       # R = E[array_signal_n * array_signal_n'] where n is for each array of partial samples possible
    r /= (length(signal) - length_filter + 1)       # r = E[array_signal_n * bpsk_symbols[n]] where n is for each array of partial samples possible

    θ = inv(R) * r  # θ is the estimation of coefficients of the filter such that the MSE between the predicted symbols and the ground truth is minimal at convergence
    
    return θ
end;

In [ ]:
θ_mse = min_mse_filter(noisy_signal, length_filter, bpsk_symbols);

In [ ]:
function distance_with_θ_mse(res, θ_mse)    # Calculates the distance between the LMS estimation of filter coeffs and min MSE coeffs at each time

    distances = zeros(length(res))

    for n in 1:length(res)
        distances[n] = norm(θ_mse - res[n])
    end

    return distances
end;

In [ ]:
distances = distance_with_θ_mse(res, θ_mse);

In [ ]:
plot(distances, xlabel="n", ylabel="Distance", title="Evolution of distance btw LMS and min MSE coeffs", label="")

<span style="color:#DAF7A6">
As we can see,  we observe that the solution of the LMS Algorithm converges rapidly to the min MSE solution (after ≈ 200 iterations). We note some fluctuations around the MSE solution after convergence, it is due to the fact that our criterion (objective function) for the LMS algorithm evolves with n. </br>
Moreover, it seems we do not converge completely to the MSE solution as there is still a residual error whereas it should in theory. This can be due to the fact that in theory, the convergence to the min MSE solution is proven with simplified hypotheses (ex: stationarity, independence, circular gaussian random variables, etc). It can also be due to the fact that for instance the stepsize chosen is not fully appropriate.

> Let us study the convergence of the filter, depending on the SNR. 

In [ ]:
SNR_values = 0:7:30
# The column i of this matrix contains the distances of LMS sol from min MSE sol at each time n for a signal with noise at SNR = SNR_values[i]
matrix_distances = zeros(length(signal)-length_filter+1, length(SNR_values)) 

for i in 1:length(SNR_values)

    noisy_signal = add_noise(signal, SNR_values[i])
    res = stoch_gradient_lms(noisy_signal, length_filter, bpsk_symbols, μ, init_θ, ∇f)
    θ_mse = min_mse_filter(noisy_signal, length_filter, bpsk_symbols)

    matrix_distances[:, i] = distance_with_θ_mse(res, θ_mse)
end;

In [ ]:
graph = plot(xlabel="n", ylabel="Distance", title="Distance btw LMS and min MSE coeffs for SNR values", label="")
for i in 1:length(SNR_values)
    plot!(matrix_distances[:, i], label="SNR $(SNR_values[i]) dB")
end
graph

<span style="color:#DAF7A6">
We observe a situation that was expected: As the SNR decreases, the prediction gets biased because we cannot eliminate completely this gaussian noise. The higher the SNR, the closer we get to the filter that minimizes the MSE. For low values of SNR, we even have some peaks after "convergence" (to the wrong solution) that increase a lot the distance to the min MSE solution for a time.

## <span style="color:#00B8DE"> Training & Testing LMS Filter </span>

> Let us train the filter with $n_s=10^4$ symbols. Then we will generate data for $n_s=10^6$ symbols and $SNR=15dB$, and let's observe the histogram of data.

In [ ]:
# Training set
ns = 10^4
bpsk_symbols = generate_bpsk_symbols(ns)    # Training set ground truth
h = [1, 0.8, 0.3]
h /= norm(h)
signal = generate_signal(bpsk_symbols, h)
SNR = 15
noisy_signal = add_noise(signal, SNR);  # Signal on which we will train our LMS algorithm

In [ ]:
length_filter = 10
max_μ = 1 / (length_filter * var(noisy_signal))
μ = 0.5 * max_μ
init_θ = randn(length_filter)

res = stoch_gradient_lms(noisy_signal, length_filter, bpsk_symbols, μ, init_θ, ∇f)
θ_lms = res[end]    # Coefficients on the filter estimated on training set

θ_mse = min_mse_filter(noisy_signal, length_filter, bpsk_symbols);

In [ ]:
ns_test = 10^6
bpsk_symbols_test = generate_bpsk_symbols(ns_test)      # Testing set ground truth
SNR_test = 15
noisy_signal_test = add_noise(generate_signal(bpsk_symbols_test, h), SNR_test)  # Signal on which we will test our LMS algorithm

histogram(noisy_signal_test, normalize=true, xlabel="Value", ylabel="Frequency", title="Histogram of Signal Received for Testing", label="")

<span style="color:#DAF7A6">
We can see that it is not a good option to predict the symbols received only based on a threshold at 0 and without the help of an algorithm. In fact, most of the recieved signal values are clustered around 0.

> Let us deconvolve the sequence, estimate the error probability $P_e$, and observe the histograms of equalizer output.

In [ ]:
deconv_signal_lms = convolve(noisy_signal_test, θ_lms)
deconv_signal_mse = convolve(noisy_signal_test, θ_mse);

In [ ]:
histo1 = histogram(deconv_signal_lms, normalize=true, title="Histogram of Deconvolved Signal with LMS", label="")
histo2 = histogram(deconv_signal_mse, normalize=true, ylabel="Frequency", title="Histogram of Deconvolved Signal with min MSE", label="")
plot(histo1, histo2, size=(1200, 400))

<span style="color:#DAF7A6">
Contrary to the previous histogram, we can see that once we have applied the LMS filter or the min MSE filter, it is much easier to separate the data between -1 and 1 as the signal values are not clustered around 0 but more around -1 and +1. It is even more visible with the min MSE filter than with the LMS filter.

In [ ]:
# The predicted symbols are estimated based on a threshold from the deconvolved signal (i.e. -1 if value > 0 and +1 otherwise)

pred_symbols_test_direct = threshold_pred_symbols(noisy_signal_test)
pred_symbols_test_lms = threshold_pred_symbols(deconv_signal_lms)
pred_symbols_test_mse = threshold_pred_symbols(deconv_signal_mse)

# Error rate if we do nothing to the signal received, and just use a threshold
error_proba_test_direct = calculate_error_proba(bpsk_symbols_test, pred_symbols_test_direct)    
# Error rate if we apply LMS filter to the signal received, and then use a threshold
error_proba_test_lms = calculate_error_proba(bpsk_symbols_test, pred_symbols_test_lms)
# Error rate if we apply min MSE filter to the signal received, and then use a threshold  
error_proba_test_mse = calculate_error_proba(bpsk_symbols_test, pred_symbols_test_mse)

println("Error Probability (Pe) on Testing Set without processing : $(round(error_proba_test_direct*100,digits=2)) %")
println("Error Probability (Pe) on Testing Set with LMS filter : $(round(error_proba_test_lms*100,digits=2)) %")
println("Error Probability (Pe) on Testing Set with min MSE filter : $(round(error_proba_test_mse*100,digits=2)) %")

<span style="color:#DAF7A6">
As we expected by looking at the histograms above, the min MSE filter is the one that performs the best in terms of error rate, closely followed by the LMS filter. As we can see, if we don't do any processing, the error rate is very high. </br>
So why don't we use each time the min MSE filter equation instead of LMS equation ? Because if we have a lower number of observations or if the data is not "toy data", then the min MSE filter would not be as good.

> Let us analyze the influence of the different parameters on the behavior of the algorithm.

In [ ]:
ns = 10^4
bpsk_symbols = generate_bpsk_symbols(ns)
h = [1, 0.8, 0.3]
h /= norm(h)
signal = generate_signal(bpsk_symbols, h)
SNR = 15
noisy_signal = add_noise(signal, SNR);
length_filter = 10
max_μ = 1 / (length_filter * var(noisy_signal))
μ = 0.5 * max_μ
init_θ = randn(length_filter);

<span style="color:#DAF7A6">
Let's first focus on the influence of the stepsize.

In [ ]:
# Influence of stepsize on convergence of LMS

μ_factors = [0.02, 0.5, 0.8, 1.5]
μ_values = μ_factors * max_μ
# The column i of this matrix contains the distances of LMS sol from min MSE sol at each time n for a stepsize at μ = μ_values[i]
matrix_distances = zeros(length(signal)-length_filter+1, length(μ_values))

θ_mse = min_mse_filter(noisy_signal, length_filter, bpsk_symbols)

for i in 1:length(μ_values)

    res = stoch_gradient_lms(noisy_signal, length_filter, bpsk_symbols, μ_values[i], init_θ, ∇f)

    matrix_distances[:, i] = distance_with_θ_mse(res, θ_mse)
end;

graph = plot(xlabel="n", ylabel="Log(Distance)", title="Log(Distance) btw LMS & min MSE coeffs for μ values", label="")
for i in 1:length(μ_values)
    plot!(log.(matrix_distances[:, i]), label="μ = $(μ_factors[i]) max μ")
end
graph

<span style="color:#DAF7A6">
Here we have plot the log of the distance to help visualize each curve.
We can see that if the stepsize is very low, the convergence of the LMS algorithm towards the min MSE solution can take some time, but it seems the convergence is better than for higher stepsizes. On contrary, if the stepsize if above the maximum stepsize required for convergence (here 50% above) we have immediately a divergence of the solution given by the LMS algorithm.

<span style="color:#DAF7A6">
The influence of the SNR has already been studied previously, so we won't focus on it. Let's focus on the length of the filter.

In [ ]:
# Influence of filter length on convergence of LMS

length_filter_values = [2, 5, 10, 20]

graph = plot(xlabel="n", ylabel="Distance", title="Distance btw LMS & min MSE coeffs for filter lengths", label="")

for i in 1:length(length_filter_values)

    θ_mse = min_mse_filter(noisy_signal, length_filter_values[i], bpsk_symbols)
    res = stoch_gradient_lms(noisy_signal, length_filter_values[i], bpsk_symbols, μ, randn(length_filter_values[i]), ∇f)

    plot!(distance_with_θ_mse(res, θ_mse), label="l_w = $(length_filter_values[i])")
end;

graph

<span style="color:#DAF7A6">
It seems that, in this case (it might not be the case with other datasets), the lower the length of the filter, the smaller the distance between the solution from LMS and the min MSE solution, but in all cases, the algorithm does not diverge.